# Tentativas de elaborar um algoritmo simbólico para realizar resumos

Este notebook apresenta algumas tentativas de elaborar um algoritmo simbólico de IA para realizar a sumarização de textos de politicas publicas.

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import collections

def summarize_text_by_word_frequency(text, num_sentences=3, language='portuguese'):
    # Tokeniza o texto em sentenças
    sentences = sent_tokenize(text, language=language)
    
    # Tokeniza o texto em palavras e remove stopwords e palavras não alfabéticas
    words = [word.lower() for word in word_tokenize(text, language=language)
             if word.isalpha() and word.lower() not in stopwords.words(language)]
    
    # Calcula a frequência de cada palavra
    word_frequencies = collections.Counter(words)
    
    # Calcula os scores de cada sentença baseando-se na soma das frequências das palavras que contém
    sentence_scores = {}
    for sentence in sentences:
        sentence_words = word_tokenize(sentence, language=language)
        score = 0
        for word in sentence_words:
            if word.lower() in word_frequencies:
                score += word_frequencies[word.lower()]
        sentence_scores[sentence] = score
    
    # Ordena as sentenças pelo score e seleciona as num_sentences mais altas
    selected_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    
    # Monta o sumário
    summary = ' '.join(selected_sentences)
    return summary

# Texto de entrada
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    example_text = file.read().strip()
    
summary = summarize_text_by_word_frequency(example_text)
print("Sumário Baseado na Frequência de Palavras:\n", summary)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import numpy as np

def summarize_text_by_tfidf(text, num_sentences=3, language='portuguese'):
    # Tokeniza o texto em sentenças
    sentences = sent_tokenize(text, language=language)

    # Cria uma instância de TfidfVectorizer, especificando a língua portuguesa para stopwords
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words(language))

    # Calcula a matriz TF-IDF para as sentenças
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    # Calcula a soma dos valores TF-IDF para cada sentença
    sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)

    # Ordena as sentenças de acordo com seus scores e seleciona as mais relevantes
    sorted_indices = np.argsort(sentence_scores)[::-1]
    selected_sentences = [sentences[idx] for idx in sorted_indices[:num_sentences]]

    # Compila o sumário
    summary = ' '.join(selected_sentences)
    return summary

# Texto de entrada
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    example_text = file.read().strip()
    
summary = summarize_text_by_tfidf(example_text)
print("Sumário Baseado em TF-IDF:\n", summary)

Sumário Baseado em TF-IDF:
 JOSÉ MUCIO MONTEIRO FILHO

Ministro de Estado da Defesa

Presidente do Conselho

ALTE ESQ MARCOS SAMPAIO OLSEN

Comandante da Marinha

GEN EX TOMÁS MIGUEL MINÉ RIBEIRO PAIVA

Comandante do Exército

TEN BRIG AR MARCELO KANITZ DAMASCENO

Comandante da Aeronáutica

ALTE ESQ RENATO RODRIGUES DE AGUIAR FREIRE

Chefe do Estado-Maior Conjunto das Forças Armadas

LUIZ HENRIQUE POCHYLY DA COSTA

Secretário-Geral do Ministério da Defesa

ANEXO

PLANO ESTRATÉGICO SETORIAL - PES 2024 - 2027

1. OSD 2 - Aprimorar o preparo das Forças Armadas para o cumprimento de sua destinação constitucional

Aperfeiçoar e construir capacidades militares de defesa que mantenham as Forças Armadas em condições de serem empregadas para defender a soberania, os interesses e a integridade do Estado brasileiro, se e quando for necessária a aplicação conjunta e sinérgica do poder militar, considerados os ambientes de emprego presentes e futuros. IDENTIDADE ESTRATÉGICA DO SETOR DE DEFESA

A Re

In [4]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from nltk.tokenize import sent_tokenize

def summarize_text_by_lsa(text, num_topics=1, num_sentences=3, language='portuguese'):
    # Tokeniza o texto em sentenças
    sentences = sent_tokenize(text, language=language)
    
    # Cria uma instância de TfidfVectorizer, especificando a língua portuguesa para stopwords
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words(language))
        
    # Calcula a matriz TF-IDF para as sentenças
    dtm = tfidf_vectorizer.fit_transform(sentences)
        
    # Configura o modelo LSA usando SVD truncado
    svd_model = TruncatedSVD(n_components=num_topics)
    lsa = make_pipeline(svd_model, Normalizer(copy=False))
    
    # Aplica LSA ao modelo DTM
    dtm_lsa = lsa.fit_transform(dtm)
    
    # Encontra as sentenças mais representativas para cada tópico
    sorted_indices = dtm_lsa[:, 0].argsort()[::-1]
    selected_sentences = [sentences[idx] for idx in sorted_indices[:num_sentences]]
    
    # Compila o sumário
    summary = ' '.join(selected_sentences)
    return summary

# Texto de entrada
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    example_text = file.read().strip()
    
summary = summarize_text_by_lsa(example_text)
print("Sumário Baseado em LSA:\n", summary)

Sumário Baseado em LSA:
 Somente o CONSUG/MD pode determinar a realização daquelas alterações, por meio de resolução do Colegiado, em conformidade com o estabelecido no art 7º da Resolução CONSUG/MD nº 8, de 16 de junho de 2021. As prioridades estabelecidas no PES 2024-2027, estando alinhadas com a cadeia de valor setorial, além de contribuir para o alcance dos objetivos setoriais de defesa, resultarão no aprimoramento e fortalecimento desses macroprocessos finalísticos. No prazo de vigência do PES, poderá haver a necessidade de realizar-se alterações no plano, em virtude de indicações originadas do processo de acompanhamento da implementação da estratégia, de indicações da análise estratégica ou como resultado da gestão de riscos.


## Primeira tentativa baseada em Regras
Neste utilizamso uma classe de sumarizador baseado em regras para aplicar algumas regras estabelecidas com base na observação dos dados coletados no corpus (arquivo combined_corpus.csv). O resultado é totalmente insatisfatório. 

In [14]:
import re
from dateutil import parser

class RuleBasedSummarizer:
    def __init__(self, rules):
        self.rules = rules

    def analyze_text(self, text):
        # Decompor texto em frases
        sentences = re.split(r'\. |\n', text)
        # Remover sentenças vazias ou que não terminam em ponto
        sentences = [sentence.strip() for sentence in sentences if sentence.strip() and sentence.endswith('.')]
        return sentences

    def apply_rules(self, sentences):
        # Aplicar regras para selecionar frases relevantes
        relevant_sentences = [s for s in sentences if any(rule(s) for rule in self.rules)]
        return relevant_sentences

    def generate_summary(self, relevant_sentences):
        # Construir o sumário
        summary = '. '.join(relevant_sentences).strip() + '.'
        return summary.replace('  ', ' ').replace('..', '.')

    def summarize(self, text):
        sentences = self.analyze_text(text)
        relevant_sentences = self.apply_rules(sentences)
        return self.generate_summary(relevant_sentences)

def normalize_date(date_str):
    try:
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=True)
        month_translation = {
            'January': 'janeiro', 'February': 'fevereiro', 'March': 'março',
            'April': 'abril', 'May': 'maio', 'June': 'junho',
            'July': 'julho', 'August': 'agosto', 'September': 'setembro',
            'October': 'outubro', 'November': 'novembro', 'December': 'dezembro'
        }
        formatted_date = parsed_date.strftime('%d de %B de %Y')
        for eng, pt in month_translation.items():
            formatted_date = formatted_date.replace(eng, pt)
        return formatted_date.capitalize()
    except ValueError:
        return date_str

def contains_decree_number(sentence):
    decree_number_pattern = re.compile(r'n[ºo]\s*\d+\.\d+')
    return bool(decree_number_pattern.search(sentence))

def contains_date(sentence):
    date_pattern = re.compile(r'\d{1,2} de [a-z]+ de \d{4}', re.IGNORECASE)
    return bool(date_pattern.search(sentence))

def contains_objective(sentence):
    objective_pattern = re.compile(r'\b(instituir|regulamentar|estabelecer|promover|implementar|criar)\b.*?\.', re.IGNORECASE)
    return bool(objective_pattern.search(sentence))

# Definir regras específicas para sumarização de políticas públicas
rules = [contains_decree_number, contains_date, contains_objective]

# Instanciar a classe RuleBasedSummarizer com as regras definidas
summarizer = RuleBasedSummarizer(rules)

# Carregar o texto de um arquivo
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    example_text = file.read().strip()

# Gerar o resumo simbólico
summary = summarizer.summarize(example_text)
print("Resumo Simbólico:\n", summary)


Resumo Simbólico:
 nº 2, de 25 de novembro de 2019, e realizar o alinhamento do planejamento estratégico com o planejamento orçamentário. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspondentes, ou seja, aqueles que receberão maior ênfase no período considerado, destacando que a priorização não significa exclusividade para o desenvolvimento ou aplicação de recursos. A Cadeia de Valor do Setor de Defesa foi aprovada pela Resolução CONSUG/MD nº 6, de 11 de novembro de 2020. O PESD 2020-2031 e, consequentemente, o PES 2024-2027, estão alinhados com importantes estratégias nacionais, destacando-se a Estratégia Nacional de Defesa (END) e a Estratégia Federal de Desenvolvimento para o Brasil no período de 2020 a 2031 - EFD 2020-2031, instituída pelo Decreto nº 10.531, de 26 de outubro de 20

Resumo Simbólico:

 nº 2, de 25 de novembro de 2019, e realizar o alinhamento do planejamento estratégico com o planejamento orçamentário. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspondentes, ou seja, aqueles que receberão maior ênfase no período considerado, destacando que a priorização não significa exclusividade para o desenvolvimento ou aplicação de recursos. A Cadeia de Valor do Setor de Defesa foi aprovada pela Resolução CONSUG/MD nº 6, de 11 de novembro de 2020. O PESD 2020-2031 e, consequentemente, o PES 2024-2027, estão alinhados com importantes estratégias nacionais, destacando-se a Estratégia Nacional de Defesa (END) e a Estratégia Federal de Desenvolvimento para o Brasil no período de 2020 a 2031 - EFD 2020-2031, instituída pelo Decreto nº 10.531, de 26 de outubro de 2020. Aprimorar o relacionamento internacional de defesa, em consonância com os interesses nacionais e a política externa, para promover a confiança mútua, a cooperação e o estreitamento de laços de amizade entre as nações, visando à solução pacífica de eventuais conflitos, além de contribuir para a paz e a segurança internacionais. As metas e os indicadores estratégicos a serem empregados para o acompanhamento da implementação do PESD 2020-2031 para o período 2024 - 2027, relacionados com os OSD priorizados neste Plano, serão os contidos no Painel de Indicadores e Metas, aprovado pela Portaria GM-MD nº 5.536, de 8 de novembro de 2022. Somente o CONSUG/MD pode determinar a realização daquelas alterações, por meio de resolução do Colegiado, em conformidade com o estabelecido no art 7º da Resolução CONSUG/MD nº 8, de 16 de junho de 2021.

## Segunda tentiva baseada apenas em Regras
Aprimoramos a primeira tentiva com base nas observações de alguns padrões inerentes aos textos que são utilizados na entrada. O resultado não é muito bom, mas aparenta constituir uma extração das ideias principais do texto com base nas regras de extração.

In [16]:
import re
from dateutil import parser

def normalize_date(date_str):
    try:
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=True)
        month_translation = {
            'January': 'janeiro', 'February': 'fevereiro', 'March': 'março',
            'April': 'abril', 'May': 'maio', 'June': 'junho',
            'July': 'julho', 'August': 'agosto', 'September': 'setembro',
            'October': 'outubro', 'November': 'novembro', 'December': 'dezembro'
        }
        formatted_date = parsed_date.strftime('%d de %B de %Y')
        for eng, pt in month_translation.items():
            formatted_date = formatted_date.replace(eng, pt)
        return formatted_date.capitalize()
    except ValueError:
        return date_str

def analyze_text(text):
    # Decompor texto em frases
    sentences = re.split(r'\. |\n', text)
    # Remover sentenças vazias ou que não terminam em ponto
    sentences = [sentence.strip() for sentence in sentences if sentence.strip() and sentence.endswith('.')]
    return sentences

def generate_summary(text):
    # Padrões para extração
    date_pattern = re.compile(r'\d{1,2} de [a-z]+ de \d{4}', re.IGNORECASE)
    decree_number_pattern = re.compile(r'n[ºo]\s*\d+\.\d+')
    # Elaborado com base na frequência do combined_corpus
    objective_pattern = re.compile(r'\b(instituir|regulamentar|estabelecer|promover|implementar|criar)\b.*?\.', re.IGNORECASE)
    
    # Extração de datas e números de decretos
    dates = date_pattern.findall(text)
    decree_numbers = decree_number_pattern.findall(text)
    
    # Seleção da data correta
    selected_date = dates[0] if dates else "Data não disponível"
    selected_date = normalize_date(selected_date)
    selected_decree_number = decree_numbers[0] if decree_numbers else "N/A"
    
    # Análise do texto para extrair sentenças
    sentences = analyze_text(text)
    
    # Extração de sentenças relevantes
    objective_sentences = [sentence for sentence in sentences if objective_pattern.search(sentence)]
    
    # Construção do resumo
    summary_sentences = [f"O Decreto {selected_decree_number}, de {selected_date}"]
    summary_sentences.extend(objective_sentences)
    
    summary = '. '.join(summary_sentences).strip() + '.'
    return summary.replace('  ', ' ').replace('..','.')

# Carregar o texto de entrada
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    example_text = file.read().strip()

# Gerar o resumo simbólico
summary = generate_summary(example_text)
print("Resumo Simbólico:\n", summary)

Resumo Simbólico:
 O Decreto nº 9.628, de 17 DE NOVEMBRO DE 2023. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspondentes, ou seja, aqueles que receberão maior ênfase no período considerado, destacando que a priorização não significa exclusividade para o desenvolvimento ou aplicação de recursos. Aprimorar o relacionamento internacional de defesa, em consonância com os interesses nacionais e a política externa, para promover a confiança mútua, a cooperação e o estreitamento de laços de amizade entre as nações, visando à solução pacífica de eventuais conflitos, além de contribuir para a paz e a segurança internacionais.


Resumo Simbólico:

 O Decreto nº 9.628, de 17 DE NOVEMBRO DE 2023. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspondentes, ou seja, aqueles que receberão maior ênfase no período considerado, destacando que a priorização não significa exclusividade para o desenvolvimento ou aplicação de recursos. Aprimorar o relacionamento internacional de defesa, em consonância com os interesses nacionais e a política externa, para promover a confiança mútua, a cooperação e o estreitamento de laços de amizade entre as nações, visando à solução pacífica de eventuais conflitos, além de contribuir para a paz e a segurança internacionais.

## Abordagem baseada na Frequência das Palavras

In [19]:
import re
from collections import Counter
from dateutil import parser
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Função para normalizar datas
def normalize_date(date_str):
    try:
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=True)
        month_translation = {
            'January': 'janeiro', 'February': 'fevereiro', 'March': 'março',
            'April': 'abril', 'May': 'maio', 'June': 'junho',
            'July': 'julho', 'August': 'agosto', 'September': 'setembro',
            'October': 'outubro', 'November': 'novembro', 'December': 'dezembro'
        }
        formatted_date = parsed_date.strftime('%d de %B de %Y')
        for eng, pt in month_translation.items():
            formatted_date = formatted_date.replace(eng, pt)
        return formatted_date.capitalize()
    except ValueError:
        return date_str

# Função para analisar texto e remover pontuações
def analyze_text(text):
    # Decompor texto em frases
    sentences = re.split(r'\. |\n', text)
    # Remover sentenças vazias ou que não terminam em ponto
    sentences = [sentence.strip() for sentence in sentences if sentence.strip() and sentence.endswith('.')]
    # Remover pontuações das sentenças
    table = str.maketrans('', '', string.punctuation)
    stripped_sentences = [sentence.translate(table) for sentence in sentences]
    return stripped_sentences, sentences

# Função para calcular a frequência das palavras
def word_frequencies(text):
    words = word_tokenize(text.lower())
    stop_words = set(stopwords.words('portuguese'))
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return Counter(words)

# Função para gerar o resumo baseado em frequências das palavras
def generate_frequency_summary(text, num_sentences=3):
    # Padrões para extração
    date_pattern = re.compile(r'\d{1,2} de [a-z]+ de \d{4}', re.IGNORECASE)
    decree_number_pattern = re.compile(r'n[ºo]\s*\d+\.\d+')
    
    # Extração de datas e números de decretos
    dates = date_pattern.findall(text)
    decree_numbers = decree_number_pattern.findall(text)
    
    # Seleção da data correta
    selected_date = dates[0] if dates else "Data não disponível"
    selected_date = normalize_date(selected_date)
    selected_decree_number = decree_numbers[0] if decree_numbers else "N/A"
    
    # Análise do texto para extrair sentenças e calcular frequências
    stripped_sentences, original_sentences = analyze_text(text)
    word_freq = word_frequencies(text)
    
    # Pontuar as sentenças
    sentence_scores = {}
    for i, sentence in enumerate(stripped_sentences):
        score = sum(word_freq[word] for word in word_tokenize(sentence.lower()) if word in word_freq)
        sentence_scores[original_sentences[i]] = score
    
    # Selecionar as sentenças mais relevantes
    sorted_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    summary_sentences = sorted_sentences[:num_sentences]
    
    # Construção do resumo
    summary_intro = f"O Decreto {selected_decree_number}, de {selected_date}"
    summary = '. '.join([summary_intro] + summary_sentences).strip() + '.'
    return summary.replace('  ', ' ').replace('..','.')

# Carregar o texto de entrada
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    example_text = file.read().strip()

# Gerar o resumo baseado em frequências das palavras
summary = generate_frequency_summary(example_text)
print("Resumo Baseado em Frequências:\n", summary)

Resumo Baseado em Frequências:
 O Decreto nº 9.628, de 17 DE NOVEMBRO DE 2023. Fortalecer o setor de ciência, tecnologia e inovação das Forças Armadas, a área de ciência e tecnologia de interesse da Defesa Nacional, contribuir para o fortalecimento da Base Industrial de Defesa (BID) e o aperfeiçoamento da integração da tríade governo-indústria-academia para o domínio e o uso de tecnologias de interesse da defesa. Missão - preparar as Forças Armadas, mantendo-as em permanente estado de prontidão para serem empregadas na defesa da Pátria, na garantia dos poderes constitucionais, na garantia da lei e da ordem; no cumprimento das atribuições subsidiárias; e em apoio à política externa; a fim de contribuir com o esforço nacional de defesa. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspond

Resumo Baseado em Frequências:

 O Decreto nº 9.628, de 17 DE NOVEMBRO DE 2023. Fortalecer o setor de ciência, tecnologia e inovação das Forças Armadas, a área de ciência e tecnologia de interesse da Defesa Nacional, contribuir para o fortalecimento da Base Industrial de Defesa (BID) e o aperfeiçoamento da integração da tríade governo-indústria-academia para o domínio e o uso de tecnologias de interesse da defesa. Missão - preparar as Forças Armadas, mantendo-as em permanente estado de prontidão para serem empregadas na defesa da Pátria, na garantia dos poderes constitucionais, na garantia da lei e da ordem; no cumprimento das atribuições subsidiárias; e em apoio à política externa; a fim de contribuir com o esforço nacional de defesa. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspondentes, ou seja, aqueles que receberão maior ênfase no período considerado, destacando que a priorização não significa exclusividade para o desenvolvimento ou aplicação de recursos.

## Abordagem baseada em TF-IDF

In [20]:
import re
from dateutil import parser
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Função para normalizar datas
def normalize_date(date_str):
    try:
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=True)
        month_translation = {
            'January': 'janeiro', 'February': 'fevereiro', 'March': 'março',
            'April': 'abril', 'May': 'maio', 'June': 'junho',
            'July': 'julho', 'August': 'agosto', 'September': 'setembro',
            'October': 'outubro', 'November': 'novembro', 'December': 'dezembro'
        }
        formatted_date = parsed_date.strftime('%d de %B de %Y')
        for eng, pt in month_translation.items():
            formatted_date = formatted_date.replace(eng, pt)
        return formatted_date.capitalize()
    except ValueError:
        return date_str

# Função para analisar texto e remover pontuações
def analyze_text(text):
    # Decompor texto em frases
    sentences = re.split(r'\. |\n', text)
    # Remover sentenças vazias ou que não terminam em ponto
    sentences = [sentence.strip() for sentence in sentences if sentence.strip() and sentence.endswith('.')]
    return sentences

# Função para calcular o TF-IDF das sentenças
def calculate_tfidf(sentences):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    return tfidf_matrix, vectorizer

# Função para pontuar as sentenças com base no TF-IDF
def score_sentences(tfidf_matrix, vectorizer):
    sentence_scores = tfidf_matrix.sum(axis=1).A1  # Soma dos valores TF-IDF de cada sentença
    return sentence_scores

# Função para gerar o resumo baseado em TF-IDF
def generate_tfidf_summary(text, num_sentences=3):
    # Padrões para extração
    date_pattern = re.compile(r'\d{1,2} de [a-z]+ de \d{4}', re.IGNORECASE)
    decree_number_pattern = re.compile(r'n[ºo]\s*\d+\.\d+')
    
    # Extração de datas e números de decretos
    dates = date_pattern.findall(text)
    decree_numbers = decree_number_pattern.findall(text)
    
    # Seleção da data correta
    selected_date = dates[0] if dates else "Data não disponível"
    selected_date = normalize_date(selected_date)
    selected_decree_number = decree_numbers[0] if decree_numbers else "N/A"
    
    # Análise do texto para extrair sentenças
    sentences = analyze_text(text)
    
    # Calcular TF-IDF das sentenças
    tfidf_matrix, vectorizer = calculate_tfidf(sentences)
    
    # Pontuar as sentenças
    sentence_scores = score_sentences(tfidf_matrix, vectorizer)
    
    # Selecionar as sentenças mais relevantes
    scored_sentences = list(zip(sentences, sentence_scores))
    sorted_sentences = sorted(scored_sentences, key=lambda x: x[1], reverse=True)
    summary_sentences = [sentence for sentence, score in sorted_sentences[:num_sentences]]
    
    # Construção do resumo
    summary_intro = f"O Decreto {selected_decree_number}, de {selected_date}"
    summary = '. '.join([summary_intro] + summary_sentences).strip() + '.'
    return summary.replace('  ', ' ').replace('..','.')

# Carregar o texto de entrada
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    example_text = file.read().strip()

# Gerar o resumo baseado em TF-IDF
summary = generate_tfidf_summary(example_text)
print("Resumo Baseado em TF-IDF:\n", summary)


Resumo Baseado em TF-IDF:
 O Decreto nº 9.628, de 17 DE NOVEMBRO DE 2023. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspondentes, ou seja, aqueles que receberão maior ênfase no período considerado, destacando que a priorização não significa exclusividade para o desenvolvimento ou aplicação de recursos. Sustentação - aptidão para prover o apoio necessário às operações militares e às atividades inerentes ao cumprimento das atribuições de determinada força, compreendida como o conjunto de unidades integradas por navios, aeronaves, tropas ou pela combinação desses componentes, que dispõe de comando próprio e é estabelecida para fins administrativos, operacionais ou logísticos. Estimular a realização de estudos e debates sobre temas ligados à defesa nacional no setor de defesa e na socied

Resumo Baseado em TF-IDF:

 O Decreto nº 9.628, de 17 DE NOVEMBRO DE 2023. As limitações de recursos fazem com que o Conselho Superior de Governança do Ministério da Defesa (CONSUG/MD) tenha de estabelecer, para o ciclo de gestão 2024-2027, a priorização dos OSD, das Iniciativas Estratégicas de Defesa (IED) e métricas correspondentes, ou seja, aqueles que receberão maior ênfase no período considerado, destacando que a priorização não significa exclusividade para o desenvolvimento ou aplicação de recursos. Sustentação - aptidão para prover o apoio necessário às operações militares e às atividades inerentes ao cumprimento das atribuições de determinada força, compreendida como o conjunto de unidades integradas por navios, aeronaves, tropas ou pela combinação desses componentes, que dispõe de comando próprio e é estabelecida para fins administrativos, operacionais ou logísticos. Estimular a realização de estudos e debates sobre temas ligados à defesa nacional no setor de defesa e na sociedade brasileira, particularmente no meio acadêmico, bem como a inserção do assunto no sistema educacional do país, com o propósito de ampliar o conhecimento de assunto que afeta a existência da nação como ente soberano e respeitado na comunidade internacional.